[View in Colaboratory](https://colab.research.google.com/github/maddy46/mlblr4a_experiments/blob/master/4B_May27_VA84.ipynb)

In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
!rm utils.py
!wget https://raw.githubusercontent.com/Zahlii/colab-tf-utils/master/utils.py
import utils
import os
import keras

def compare(best, new):
  return best.losses['val_acc'] < new.losses['val_acc']

def path(new):
  if new.losses['val_acc'] > 0.2:
    return '4B_008_%s.h5' % new.losses['val_acc']

callbacks = cb = [
      utils.GDriveCheckpointer(compare,path)
]

--2018-05-27 10:30:03--  https://raw.githubusercontent.com/Zahlii/colab-tf-utils/master/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6935 (6.8K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   6.77K  --.-KB/s    in 0s      

2018-05-27 10:30:03 (61.3 MB/s) - ‘utils.py’ saved [6935/6935]



In [0]:
# Hyperparameters
batch_size = 50
num_classes = 10
epochs =20
#l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
print (img_height)
print (img_width)

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

32
32


In [0]:
print (x_train.shape[0])
print (x_test.shape[0])

50000
10000


In [0]:
new_images = np.empty((50000, 24, 24, 3), dtype='uint8')
new_test = np.empty((10000, 24, 24, 3), dtype='uint8')

for image_count in range(x_train.shape[0]):
    new_images[image_count] = cv2.resize(x_train[image_count], (24, 24))
for image_count1 in range(x_test.shape[0]):
    new_test[image_count1] = cv2.resize(x_test[image_count1], (24, 24))
img_height1, img_width1, channel1 = new_images.shape[1],new_images.shape[2],new_images.shape[3]
print (img_height1)
print (img_width1)

print('x Processing Done.')

24
24
x Processing Done.


In [0]:
print (new_images.shape)
print (y_train.shape)
print (new_test.shape)
print (y_test.shape)

(50000, 24, 24, 3)
(50000, 10)
(10000, 24, 24, 3)
(10000, 10)


In [0]:
datagen = ImageDataGenerator(zoom_range=0.2, rotation_range=20)
datagen.fit(x_train)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 16
dropout_rate = 0.2
l = 16
input = Input(shape=(img_height, img_width, channel,))
print(channel)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)
#output = output_layer(First_Block)


3


In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_273 (Conv2D)             (None, 32, 32, 16)   432         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_273 (BatchN (None, 32, 32, 16)   64          conv2d_273[0][0]                 
__________________________________________________________________________________________________
activation_273 (Activation)     (None, 32, 32, 16)   0           batch_normalization_273[0][0]    
__________________________________________________________________________________________________
conv2d_274

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:

model.fit(new_images, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(new_test, y_test),callbacks=cb)

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
34200/50000 [===================>..........] - ETA: 1:19 - loss: 1.7327 - acc: 0.3490

50000/50000 [==============================] - 255s 5ms/step - loss: 1.6479 - acc: 0.3841 - val_loss: 1.5614 - val_acc: 0.4559


Uploading file 4B_008_0.4558999978005886.h5 to folder Colab Notebooks: 100%|██████████| 100/100 [00:00<00:00, 110.95it/s]


Epoch 2/20
50000/50000 [==============================] - 233s 5ms/step - loss: 1.3056 - acc: 0.5242 - val_loss: 1.2291 - val_acc: 0.5664
Removing old cloud file 4B_008_0.4558999978005886.h5


Uploading file 4B_008_0.5664000003039837.h5 to folder Colab Notebooks: 100%|██████████| 100/100 [00:00<00:00, 148.75it/s]


...

50000/50000 [==============================] - 232s 5ms/step - loss: 0.5918 - acc: 0.7937 - val_loss: 0.8687 - val_acc: 0.7379
No improvement.
Epoch 18/20
 4300/50000 [=>............................] - ETA: 3:20 - loss: 0.5614 - acc: 0.8019

50000/50000 [==============================] - 232s 5ms/step - loss: 0.5797 - acc: 0.7972 - val_loss: 0.6581 - val_acc: 0.7909
Removing old cloud file 4B_008_0.7674999979138374.h5


Uploading file 4B_008_0.7908999955654145.h5 to folder Colab Notebooks: 100%|██████████| 100/100 [00:00<00:00, 137.71it/s]


Epoch 19/20
50000/50000 [==============================] - 232s 5ms/step - loss: 0.5629 - acc: 0.8021 - val_loss: 0.7691 - val_acc: 0.7646
No improvement.
Epoch 20/20
  600/50000 [..............................] - ETA: 3:40 - loss: 0.5497 - acc: 0.8167

50000/50000 [==============================] - 232s 5ms/step - loss: 0.5529 - acc: 0.8077 - val_loss: 0.6630 - val_acc: 0.7893
No improvement.


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("saved_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
!ls -lrt
!date

total 19856
-rwxr-xr-x 1 root root 16117632 Jul 15  2017 ngrok
-rw-r--r-- 1 root root     5214 May 27 10:16 tboard.py
drwxr-xr-x 2 root root     4096 May 27 10:16 __pycache__
drwxr-xr-x 2 root root     4096 May 27 10:16 log
drwxr-xr-x 1 root root     4096 May 27 10:16 datalab
-rw-r--r-- 1 root root     6935 May 27 10:30 utils.py
-rw-r--r-- 1 root root  2154152 May 27 11:53 4B_008_0.7908999955654145.h5
-rw-r--r-- 1 root root  2022968 May 27 12:01 saved_model.h5
Sun May 27 12:07:59 UTC 2018


In [0]:
#from google.colab import files

#files.download('DNST_model.h5')

In [0]:
print (x_train.shape)
print(x_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [0]:
num_filter = 16
dropout_rate = 0.2
l = 16
inputs = Input(shape=(img_height, img_width, channel,))
print(channel)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)
#output = output_layer(First_Block)

3


In [0]:

#model = Model(inputs=[inputs], outputs=[output])
model.load_weights('saved_model.h5',by_name=True)
epochss =30

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochss,
                    verbose=1,
                    validation_data=(x_test, y_test),callbacks=cb)


Train on 50000 samples, validate on 10000 samples
Epoch 1/30
34150/50000 [===================>..........] - ETA: 1:55 - loss: 1.6874 - acc: 0.3693

50000/50000 [==============================] - 376s 8ms/step - loss: 1.5842 - acc: 0.4114 - val_loss: 1.7178 - val_acc: 0.4576
No improvement.
Epoch 2/30
15250/50000 [========>.....................] - ETA: 3:49 - loss: 1.2658 - acc: 0.5386

50000/50000 [==============================] - 352s 7ms/step - loss: 1.1912 - acc: 0.5694 - val_loss: 1.3218 - val_acc: 0.5601
No improvement.
Epoch 3/30
 8500/50000 [====>.........................] - ETA: 4:34 - loss: 1.0693 - acc: 0.6121

50000/50000 [==============================] - 351s 7ms/step - loss: 1.0312 - acc: 0.6293 - val_loss: 1.4254 - val_acc: 0.5505
No improvement.
Epoch 4/30
 6100/50000 [==>...........................] - ETA: 4:49 - loss: 0.9784 - acc: 0.6487

50000/50000 [==============================] - 351s 7ms/step - loss: 0.9387 - acc: 0.6669 - val_loss: 1.1552 - val_acc: 0.6316
No improvement.
Epoch 5/30
 5250/50000 [==>...........................] - ETA: 4:54 - loss: 0.8585 - acc: 0.6872

...

Uploading file 4B_008_0.8161999955773354.h5 to folder Colab Notebooks: 100%|██████████| 100/100 [00:01<00:00, 91.34it/s]


Epoch 28/30
50000/50000 [==============================] - 345s 7ms/step - loss: 0.3962 - acc: 0.8627 - val_loss: 0.5887 - val_acc: 0.8224
Removing old cloud file 4B_008_0.8161999955773354.h5


Uploading file 4B_008_0.822399994134903.h5 to folder Colab Notebooks: 100%|██████████| 100/100 [00:00<00:00, 118.73it/s]


Epoch 29/30
50000/50000 [==============================] - 348s 7ms/step - loss: 0.3929 - acc: 0.8602 - val_loss: 0.6546 - val_acc: 0.8087
No improvement.
Epoch 30/30
  550/50000 [..............................] - ETA: 5:20 - loss: 0.3459 - acc: 0.8855

50000/50000 [==============================] - 347s 7ms/step - loss: 0.3874 - acc: 0.8628 - val_loss: 0.6415 - val_acc: 0.8153
No improvement.


In [0]:
model.load_weights('saved_model.h5',by_name=True)
epochss =30
'''
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochss,
                    verbose=1,
                    validation_data=(x_test, y_test),callbacks=cb)
'''
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochss, 
                    verbose=1, validation_data=(x_test, y_test),
                    steps_per_epoch=len(x_train)/batch_size, callbacks=cb)

Epoch 1/30
1000/1000 [==============================] - 360s 360ms/step - loss: 0.6260 - acc: 0.7819 - val_loss: 0.7366 - val_acc: 0.7843
No improvement.
Epoch 2/30
  24/1000 [..............................] - ETA: 5:28 - loss: 0.5138 - acc: 0.8225

1000/1000 [==============================] - 359s 359ms/step - loss: 0.5957 - acc: 0.7902 - val_loss: 0.8158 - val_acc: 0.7667
No improvement.
Epoch 3/30
  79/1000 [=>............................] - ETA: 5:12 - loss: 0.5637 - acc: 0.8028

1000/1000 [==============================] - 361s 361ms/step - loss: 0.5902 - acc: 0.7923 - val_loss: 0.6170 - val_acc: 0.8139
No improvement.
Epoch 4/30
  99/1000 [=>............................] - ETA: 5:06 - loss: 0.5559 - acc: 0.8069

1000/1000 [==============================] - 360s 360ms/step - loss: 0.5758 - acc: 0.7982 - val_loss: 0.8708 - val_acc: 0.7609
No improvement.
Epoch 5/30
 106/1000 [==>...........................] - ETA: 4:59 - loss: 0.5464 - acc: 0.8106

1000/1000 [==============================] - 358s 358ms/step - loss: 0.5658 - acc: 0.8033 - val_loss: 0.6625 - val_acc: 0.8041
No improvement.
Epoch 6/30
 108/1000 [==>...........................] - ETA: 4:59 - loss: 0.5822 - acc: 0.7943

...

Epoch 27/30
1000/1000 [==============================] - 359s 359ms/step - loss: 0.4770 - acc: 0.8323 - val_loss: 0.5829 - val_acc: 0.8332
No improvement.
Epoch 28/30
  19/1000 [..............................] - ETA: 5:27 - loss: 0.4919 - acc: 0.8211

1000/1000 [==============================] - 360s 360ms/step - loss: 0.4736 - acc: 0.8343 - val_loss: 0.5341 - val_acc: 0.8376
No improvement.
Epoch 29/30
  77/1000 [=>............................] - ETA: 5:10 - loss: 0.4577 - acc: 0.8423

1000/1000 [==============================] - 359s 359ms/step - loss: 0.4612 - acc: 0.8391 - val_loss: 0.8172 - val_acc: 0.7774
No improvement.
Epoch 30/30
  97/1000 [=>............................] - ETA: 5:04 - loss: 0.4663 - acc: 0.8320

1000/1000 [==============================] - 359s 359ms/step - loss: 0.4682 - acc: 0.8359 - val_loss: 0.5245 - val_acc: 0.8430
Removing old cloud file 4B_008_0.8420999947190285.h5


Uploading file 4B_008_0.842999995648861.h5 to folder Colab Notebooks: 100%|██████████| 100/100 [00:00<00:00, 130.93it/s]
